# Modèles à variable dépendante limitée

Cette application reprend la proposition de Stock et Watson (le manuel) qui utilisent les données de l'article de Alicia H Munnell, Geoffrey MB Tootell, Lynn E Browne, and James McEneaney : "Mortgage lending in
boston: Interpreting hmda data." *The American Economic Review*, pages 25-53, 1996.

Dans cet article, les chercheurs de la Federal Reserve Bank de Boston souhaitent savoir les noirs américains sont discriminés dans l'accès au crédit immobilier. Ils collectent des données sur les demandes de prêts immobiliers en 1990 dans Boston et sa banlieue. Ils obtiennent 2380 observations de demandeurs blanc et noir-américain. 

Les données sont décrites dans le codebook simplifié : **TD6_codebook.png**.

## Intuition
On ouvre la base de données : **TD6_hdma_v12.dta**. 

On a la variable 'black' numérique qui nous donne l'ethnie ; à partir de cette variable on crée une variable facteur 'blackcategory' pour définir dans R que cette variable n'est une variable continue mais une variable catégorielle qui vaut "White" si black=0, "Black" si black=1. 

NB: la plupart du temps on utilisera la variable catégorielle 'hdma$blackcategory', mais pour calculer les effets marginaux à la moyenne à la main on aura besoin de la porportion de Noirs américains', et on utilisera exceptionnellement la numérique: 'black'.

Les noirs-américains (*black*) ont-ils effectivement une probabilité plus importante de voir leur demande de prêt rejetée (*deny*) ? 

*Indice : proposez un test d'égalité de moyennes ou une régression linéaire simple.*

In [ ]:
#test d'égalité de moyennes
#

In [ ]:
#régression linéaire simple
#lpm1 <- 
summary(lpm1)

Grâce à des tests d'égalité de moyennes sur les vraiables prédéterminées : *selfemp, single, hischl, probunmp*, déterminez si les blancs-américains sont de bons contrefactuels pour les noirs-américains. Peut-on donc interpréter la différence de traitement de la question précédente comme de la discrimination ?

## 1. Modèle de probabilité linéaire 
On s'intéresse à contrôler en plus pour l'effet du ratio mensualité de l'emprunt / revenu total *pi_rat* sur la probabilité de rejet. Pourquoi ? Que s'attend-on à trouver?

Rappeler ce qu'est un modèle de probabilité linéaire.

Estimer et interpréter.

## 2. Modèle de régression probit


On estime le modèle de régression probit de *deny* sur *black* et *pi_rat* par maximum de vraisemblance : **glm( y~x, binomial("probit"), data=... )**. Quel est le principe de l'estimation par maximum de vraisemblance ? Interpréter les résultats.

In [ ]:
probit1<-

Pourquoi l'interprétation des résultats est-elle compliquée ? 

#### 1. Effet marginal à la moyenne 
Rappeler ce qu'est l'effet marginal à la moyenne.

En vous aidant du code ci-dessous, calculez l'effet marginal du fait d'être noir-américain sur la probabilité de rejet. Pour cela, calculez d'abord la probabilité de rejet de l'individu moyen noir, puis de l'individu moyen blanc. Calculez ensuite la différence.

*Indice: La fonction de répartition de la loi normale s'écrit :* **pnorm()**. Les coefficients de la régression probit1 sont stockés dans le vecteur **coef(probit1)**.

In [ ]:
#Voir coef probit1

#Prob for blacks at means

#Prob for whites at means

#Difference in probs


Sur un modèle similaire, calculez maintenant l'effet marginal du fait d'avoir un ratio d'endettement de 10 points de pourcentage plus élevé pour l'individu moyen. Pour ce faire calculez la proba de rejet de l'individu moyen, puis de l'individu moyen avec 0.1 de ratio d'endettement supplémentaire. Calculez ensuite la différence.

#### 2. Effet marginal moyen

Rappeler ce qu'est l'effet marginal moyen.





Que mesure ce code ? 

*Indice: La fonction de densité de la loi normale s'écrit :* **dnorm()**

In [ ]:
hdma$gg<-dnorm(coef(probit1)[1] + coef(probit1)[2]*hdma$black2 + coef(probit1)[3]*hdma$pi_rat)
scalefactor<-mean(hdma$gg)
scalefactor

Calculer l'effet marginal moyen du ratio d'endettement. 

<font color=red> NOTE : On n'a jamais besoin de faire ces calculs à la main ! R calcule pour vous les effets marginaux moyens grâce au package 'margins'. 

Vous pouvez essayer et retrouver les résultats de pi_rat en tapant: 

*install.packages("margins")*

*library(margins)*
*AME<-margins(probit1)*
*summary(AME)*

Attention, dans votre régression, si la variable est un facteur (une variable catégorielle), il doit bien être défini comme tel ! Sinon l'effet marginal à la moyenne sera mal calculé car R appliquera la méthode des variables continues (et non des variables discrètes, alors que la variable catégorielle est discrète). 
</font>

In [ ]:
#NOTE : on peut calculer l'effet marginal très simplement avec margins :
install.packages("margins")
library(margins)
AME<-margins(probit1)
summary(AME)

## 3. Comparaison des deux modèles de régression

Calculer la probabilité de rejet quand le demandeur est blanc avec un taux d'endettement de 0.1 selon le modèle de probabilité linéaire et le modèle probit. Commentez. 



Que réprésentent les commandes et le graphique ci-dessous ?

In [ ]:
library(ggplot2)
hdma$p_rejet <- predict(probit1, hdma, type = "response")
hdma$y_hat <- predict(lpm2, hdma, type = "response")
ggplot(hdma, aes(x = pi_rat, y = p_rejet, colour = black))+ geom_point()+geom_line(aes(y = y_hat, colour = black))

## 4. Modèle de régression probit complet

Quelles sont les autres variables que vous souhaiteriez ajouter à la régression ? 

Sous quelle condition estimez-vous l'effet causal du traitement ?

Estimer le modèle probit de *black* sur *deny* en ajoutant 1 puis 2 puis 3 catégories de variables : 
- emprunt : pi_rat + hse_inc + loan_val 
- emprunteur : ccred + mcred + pubrec + denpmi
- individuelles : selfemp + single + hischl + probunmp

Afficher directement les effets marginaux en utilisant **summary(margins(** *objet estimé par glm()*)**)**. Interpréter les résultats.

Qu'ajouteriez-vous à la régression pour savoir si le ratio d'endettement a un effet différent sur la probabilité de voir son dossier rejeté par la banque selon si on est noir ou blanc ?

Voici le pseudoR2 de McFadden pour le modèle simple et multiple. Commentez la qualité générale des régressions.

In [ ]:
pR2_1 <- 1 - probit1$deviance / probit1$null.deviance 
pR2_2 <- 1 - probit2$deviance / probit2$null.deviance 
pR2_3 <- 1 - probit3$deviance / probit3$null.deviance 
pR2_4 <- 1 - probit4$deviance / probit4$null.deviance 
pR2_1
pR2_2
pR2_3
pR2_4